In [10]:
import numpy as np
import pandas as pd
import sympy as sp
from fractions import Fraction

In [96]:
NUM_PLAYERS = 3
PLAYERS = list(range(1, NUM_PLAYERS+1))
# all possible (giver, receiver) pairs
OUTCOME_PROB = Fraction(1, NUM_PLAYERS * (NUM_PLAYERS-1))  # 1/6
CANDIDATE_MOVES = [(g, r) for idx, g in enumerate(PLAYERS) for r in PLAYERS if g != r]

In [148]:
def applyMove(initState, move):
    g, r = move
    initWealth = list(initState)
    transferAmt = min(initWealth[g-1], initWealth[r-1])  # 0-indexed
    initWealth[g-1] -= transferAmt  # give
    initWealth[r-1] += transferAmt  # receive
    return tuple(initWealth)


def getChildStates(initState):
    """
    Applies all possible moves to the initial state.
    Returns the child states.
    """
    return [applyMove(initState, move) for move in CANDIDATE_MOVES]

def getCoef(initState, loser=1):
    """
    Returns the coefficients for a first-step analysis equation,
    where the hitting probability of the initial state is the subject.
    """
    # assign all states the individual outcome probability
    childStates = getChildStates(initState)
    stateCoefs = {s:OUTCOME_PROB for s in childStates}
    const = 0
    
    for state in stateCoefs.copy():
        # prune the 0 hitting prob states
        if zeroHittingProb(state, loser):
            stateCoefs.pop(state)
        # convert the hitting prob 1 states to constants
        elif alreadyHit(state, loser):
            const += stateCoefs[state]
            stateCoefs.pop(state)
        # optimsation 1: convert equal wealth states to constants
        elif equalWealth(state):
            const += stateCoefs[state] / NUM_PLAYERS
            stateCoefs.pop(state)
    
    # optimization 2: combine symmetrical states
    copy = list(stateCoefs.copy().keys())
    n = len(copy)
    for i in range(n):
        for j in range(i+1, n):
            s1, s2 = copy[i], copy[j]
            if (symmetricalStates(s1, s2)):
                combinedProb = stateCoefs.pop(s1) + stateCoefs.pop(s2)
                stateCoefs[standardiseState(s1)] = combinedProb
    
    return standardiseStates(stateCoefs), const

def zeroHittingProb(state, loser):
    """
    Returns true if the given state cannot reach the state of having the specified loser, false otherwise.
    
    """
    return (0 in state) and (state[loser-1] > 0)

def alreadyHit(state, loser):
    """
    Returns true if the given state reaches the state of having the specified loser wp1, false otherwise.
    
    """
    return state[loser-1] == 0

def symmetricalStates(s1, s2):
    return s1[0] == s2[0] and set(s1[1:]) == set(s2[1:])

def standardiseState(s):
    # convention: keep the smaller value at the front
    return s if s[1] <= s[2] else (s[0], s[2], s[1])

def standardiseStates(stateCoefs):
    return {standardiseState(s):p for s, p in stateCoefs.items()}
    
def equalWealth(s):
    return s[0] == s[1] and s[1] == s[2]

In [170]:
from sympy import *
from sympy.solvers import solve

In [150]:
def getEquation(initState, stateCoefs, const):
    # equation will be in the form of human readable equalities (like how we normally write down)
    symbols = [encodeState(s) for s in stateCoefs]
    eqn = Eq(encodeState(initState), getExpr(stateCoefs, const))
    return eqn, symbols
    
def encodeState(state):
    return Symbol(f'h{state}')

def getExpr(stateCoefs, const):
    expr = const
    for s in stateCoefs:
        expr = Add(expr, stateCoefs[s] * encodeState(s))
    return expr

In [180]:
class FirstStepAnalysis():
    def __init__(self, initState):
        self.initState = initState
        self.generatedStates = set([standardiseState(initState)])
        self.equations = []
        self.symbols = set() # or can encode all generated states at once in the end
    
    def getEquations(self):
        self.recurseGenerateEqns(self.initState)
        return self.equations
        
    def recurseGenerateEqns(self, initState):
        childStatesProb, const = getCoef(initState)  # state:prob pairs, under unified representation
        newStates = set(childStatesProb).difference(self.generatedStates)
        
        # compute for the current state
        # do this before the recursion to close off the loop
        eqn, symbols = getEquation(initState, childStatesProb, const)
        self.equations.append(eqn)

        # base case: no new states
        if not newStates:
            return

        # 
        self.symbols = self.symbols.union(symbols)
        self.generatedStates = self.generatedStates.union(newStates)
        
        # recurse
        for state in newStates:
            self.recurseGenerateEqns(state)
    
    def solveEqns(self):
        self.getEquations()
        return solve(self.equations, self.symbols)
    
    #def exportEqns(self):

In [178]:
init = (1,2,3)
test = FirstStepAnalysis(init)
eqns = test.getEquations()
test.solveEqns()

{h(1, 1, 4): 44/93,
 h(1, 2, 3): 113/217,
 h(2, 1, 3): 69/217,
 h(3, 1, 2): 5/31,
 h(4, 1, 1): 5/93}